# Occupancy Forecasting Notebook

## !!This is where the series work begins!!

## 0. Train and Test split

## 1. Occupancy Forecasting

In [ ]:
# check out some of the papers in the review
# start with some traditional methods

## 2. Count Forecasting
### Try more general approach if 1. is too hard, gather literature on count forecasting